In [2]:
from opencage.geocoder import OpenCageGeocode
from opencage.geocoder import InvalidInputError, RateLimitExceededError, UnknownError
import pandas as pd
from pprint import pprint

In [68]:
# key = '2c67f66fd0cd44cdacf36b54154b9cdf'
# key = '90f7c1eb939440d2b00dea098f80d76d'
key = 'ded082443a7c41e689f5914b123889be'
geocoder = OpenCageGeocode(key)

In [69]:
# df = pd.read_csv("data/eclipse_data_enriched_5000_years.csv")
df = pd.read_csv("data/geo_coded_data_clean.csv")

In [72]:
def reverse_geocode(row):  
    latitude = row['Eclipse Latitude']
    longitude = row['Eclipse Longitude'] 
    try:
        results = geocoder.reverse_geocode(latitude, longitude)
        if results and len(results):
            return get_location(results)
    except RateLimitExceededError as ex:
        print("Rate Limit Error")
        return str(ex)
    except InvalidInputError as ex:
        print("Invalid Input Error")
        
        return str(ex)
    
    return None, None, None

def get_location(results):
    location = results[0]['components']

    country_name = location.get('country', None)
    alpha_2_code = location.get('ISO_3166-1_alpha-2', None)
    alpha_3_code = location.get('ISO_3166-1_alpha-3', None)
    
    return country_name, alpha_2_code, alpha_3_code 


def update_csv(df, filename, batch_size=100):
    for i in range(690, 2392, batch_size):
        batch = df[i:i+batch_size]
        batch['Country'], batch['Alpha2'], batch['Alpha3'] = zip(*batch.apply(reverse_geocode, axis=1))

        batch.to_csv(filename, mode='a', header=not i, index=False)

In [73]:
update_csv(df, 'data/geo_coded_data2.csv', batch_size=100)

C:\Users\dell\AppData\Local\Temp\ipykernel_13744\1993931186.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['Country'], batch['Alpha2'], batch['Alpha3'] = zip(*batch.apply(reverse_geocode, axis=1))


In [2]:
df_with_country = pd.read_csv("data/geo_coded_data.csv")

In [15]:
df_with_country.to_csv("data/geo_coded_data.csv", index = False)

CREATE JSON WITH COUNTRY NAME TO ALPHA3 CODE MAPPING

In [32]:
import json

# Load the JSON data
with open('data/world.geojson', 'r') as f:
    data = json.load(f)

# Create a dictionary to store the mapping
country_mapping = {}

# Iterate through the features
for feature in data['features']:
    country_name = feature['properties']['name']
    country_id = feature['id']
    country_mapping[country_name] = country_id

# Write the mapping to a new JSON file
with open('data/alpha3Codes.json', 'w') as f:
    json.dump(country_mapping, f, indent=4)


ADDING CODES TO DATA

In [36]:
data = pd.read_csv("data/geo_coded_data_clean.csv")

In [37]:
import json

# Read the JSON file
with open("data/alpha3Codes.json", "r") as f:
    country_mapping = json.load(f)
    

# Function to get Alpha3Code
def get_alpha3code(country_name):
    alpha3code = country_mapping.get(country_name, "NOT FOUND")
    if alpha3code == "NOT FOUND":
        print(f"Country not found: {country_name}")
    return alpha3code

In [ ]:
# Add Alpha3Code column
data["Alpha3"] = data["Country"].apply(lambda x: get_alpha3code(x))

CLEAN GEOCODED DATA

In [3]:
data = pd.read_csv("data/geo_coded_data2.csv")

In [26]:
data.iloc[2488]

Catalog Number                                  7545
Calendar Date                           1172 June 23
Eclipse Time                                04:47:38
Delta T (s)                                      813
Lunation Number                               -10235
Saros Number                                      94
Eclipse Type                                       P
Gamma                                         1.1539
Eclipse Magnitude                           0.647739
Latitude                                       65.1N
Longitude                                      90.0W
Sun Altitude                                       0
Sun Azimuth                                      340
Path Width (km)                                  NaN
Central Duration                                 NaN
Date Time                               1172 June 23
Year                                            1172
Month                                              6
Day                                           

In [27]:
data2 = pd.read_csv("data/geo_coded_data_clean.csv")

In [37]:
df = pd.concat([data, data2.iloc[2490:]], ignore_index=True)

In [38]:
len(df)

3889

In [39]:
df.to_csv("data/geocoded_data.csv", index = False)